In [ ]:
import numpy as np
import tensorflow as tf

# Cargar el modelo .tflite
model_path = '/content/modelo.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Obtener los detalles de entrada y salida del modelo
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesar los datos de entrada
# No caída
input_array = np.array([-5.00000000e+00,  4.90000000e+01,  1.04501876e+01,  7.65318038e+01,
                        2.16356417e+00,  1.65796764e+00, -2.82000000e+02, -2.04000000e+02,
                        -2.51914131e+02,  1.50810467e+02,  1.40731285e+00,  1.18954658e+00,
                        -1.38000000e+02,  5.70000000e+01, -4.31162985e+01,  2.93777847e+03,
                        -1.11614534e+00,  5.22599949e-01, -1.17800000e+03,  8.97000000e+02,
                        -1.62613589e+01,  6.35344181e+04,  4.74747573e+00, -3.96820234e-01,
                        -1.36000000e+02,  2.16000000e+02,  6.61642351e+01,  1.62140529e+03,
                        5.19712627e+00,  5.96739826e-02, -1.89000000e+02,  2.24000000e+02,
                        -3.55689871e+00,  3.71291537e+03,  3.17250106e+00,  7.09854130e-01], dtype=np.float32)

# Caida
input_array = np.array([-9.30000000e+01, 6.46000000e+02,1.12031000e+02,1.51086154e+04,-9.44770646e-01,
                        4.63268597e-01,-2.61000000e+02, 2.95000000e+02,-1.23303667e+02, 1.79438201e+04,
                        -1.45769549e+00, 3.24141839e-01,-4.40000000e+02, 6.20000000e+01,-3.53193333e+01,
                        5.81477203e+03, 3.27258084e+00,-1.12334527e+00,-3.29300000e+03,6.12000000e+02,
                        -6.50823333e+01,2.04259444e+05,2.67079973e+01,-4.98249269e+00,-1.90200000e+03,
                        3.40200000e+03, 8.00613333e+01, 1.62843245e+05,3.07912103e+01,4.01983134e+00,
                        -3.50000000e+02,2.14800000e+03,3.51353333e+01, 5.47415617e+04,3.37131816e+01, 5.60611071e+00], dtype=np.float32)

print(np.shape(input_array))

input_array = np.expand_dims(input_array, axis=1)
input_array = np.expand_dims(input_array, axis=0)  # Añadir una dimensión para representar el batch
print(np.shape(input_array))

# Asignar los datos de entrada al modelo
interpreter.set_tensor(input_details[0]['index'], input_array)

# Ejecutar la inferencia
interpreter.invoke()

# Obtener los resultados de la inferencia
output_data = interpreter.get_tensor(output_details[0]['index'])
prediction_final = np.argmax(output_data)

print("Predicción:", prediction_final)
if prediction_final==0:
  print('La persona no ha sufrido caída')
elif prediction_final==1:
  print('La persona ha sufrido una caída. Activando alarma')

(36,)
(1, 36, 1)
Predicción: 1
La persona ha sufrido una caída. Activando alarma


In [ ]:
# Obtener los detalles de los tensores en el modelo
tensor_details = interpreter.get_tensor_details()

# Imprimir la arquitectura de la red neuronal
for tensor in tensor_details:
    tensor_name = tensor['name']
    tensor_shape = tensor['shape']
    print("Nombre del tensor:", tensor_name)
    print("Forma del tensor:", tensor_shape)
    print("---")


Nombre del tensor: serving_default_lstm_10_input:0
Forma del tensor: [ 1 36  1]
---
Nombre del tensor: strided_slice_2
Forma del tensor: [3]
---
Nombre del tensor: strided_slice_21
Forma del tensor: [3]
---
Nombre del tensor: strided_slice_22
Forma del tensor: [3]
---
Nombre del tensor: TensorArrayV2_1/num_elements
Forma del tensor: []
---
Nombre del tensor: TensorArrayV2Stack/TensorListStack/element_shape
Forma del tensor: [2]
---
Nombre del tensor: time
Forma del tensor: []
---
Nombre del tensor: transpose/perm
Forma del tensor: [3]
---
Nombre del tensor: strided_slice
Forma del tensor: []
---
Nombre del tensor: sequential_10/lstm_10/Read_2/ReadVariableOp
Forma del tensor: [256]
---
Nombre del tensor: sequential_10/dense_10/BiasAdd/ReadVariableOp
Forma del tensor: [2]
---
Nombre del tensor: sequential_10/lstm_10/zeros/packed/1
Forma del tensor: []
---
Nombre del tensor: sequential_10/lstm_10/zeros/Const
Forma del tensor: []
---
Nombre del tensor: sequential_10/lstm_10/strided_slice/s

In [ ]:
import numpy as np
import tensorflow as tf

# Ruta del archivo modelo.tflite
model_path = '/content/modelo.tflite'

# Cargar el modelo .tflite
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Obtener información del tensor de entrada
input_details = interpreter.get_input_details()[0]
input_shape = input_details['shape']
input_dtype = input_details['dtype']

# Obtener información del tensor de salida
output_details = interpreter.get_output_details()[0]
output_shape = output_details['shape']
output_dtype = output_details['dtype']

# Leer el archivo .tflite como bytes
with open(model_path, 'rb') as f:
    model_bytes = f.read()

# Generar el código C++ del archivo .cpp
cpp_code = f"""\
// Archivo del modelo generado automáticamente
// para utilizar con Arduino y TensorFlow Lite

#include <Arduino.h>

const unsigned char model[] PROGMEM = {{
    {', '.join(str(b) for b in model_bytes)}
}};

const int input_shape[] = {{{', '.join(str(dim) for dim in input_shape)}}};
const int output_shape[] = {{{', '.join(str(dim) for dim in output_shape)}}};

void setup() {{
    Serial.begin(115200);
    delay(2000);
}}

void loop() {{
    // Código de inferencia con el modelo aquí
}}
"""

# Guardar el código C++ en un archivo
cpp_file_path = '/content/modelo.cpp'
with open(cpp_file_path, 'w') as f:
    f.write(cpp_code)

print("Archivo .cpp generado con éxito:", cpp_file_path)


Archivo .cpp generado con éxito: /content/modelo.cpp
